In [1]:
import overpass
import pandas as pd
from rich import print
api = overpass.API()

In [2]:
def get_amenities(city, amenity):
    return \
        f"""
        area[name="{city}"];
        node["amenity"="{amenity}"](area);
        out;
        """

# Get Bike data from API

In [4]:
from collections import defaultdict
# Default dict of coordinates, default datatype is a list
coords = defaultdict(list)
archive = defaultdict(list)

In [5]:
for city in ["Chicago", "Denver", "Des Moines"]:
    response = api.get(get_amenities(city, 'bicycle_rental'))
    for feature in response['features']:
        try:
            coords[city].append(feature['geometry']['coordinates'])
            archive[city].append(feature)
        except:
            continue
    print(f"{city} processed")

Chicago processed

Denver processed

Des Moines processed

In [7]:
for city in ["Chicago", "Denver", "Des Moines"]:
    print(len(coords[city]))

1632

78

42

In [9]:
df = pd.DataFrame(columns=['City', 'Lat', 'Lon', 'Zipcode'])
for k,v in coords.items():
    for lon, lat in v:
        df.loc[len(df)] = [k, lat, lon, '']

In [10]:
df.head()

,City,Lat,Lon,Zipcode
0,Chicago,41.892570,-87.614492,
1,Chicago,41.877761,-87.654822,
2,Chicago,41.899624,-87.668393,
3,Chicago,41.879248,-87.640007,
4,Chicago,41.878391,-87.641006,


# Process bike coordinates data to longititudes and latitudes

In [14]:
from zipcode import get_zipcode
from tqdm.auto import tqdm
tqdm.pandas()

In [12]:
def proc_zipcode(row):
    return get_zipcode(float(row['Lat']), float(row['Lon']))

In [15]:
df['Zipcode'] = df.progress_apply(proc_zipcode, axis=1)

  0%|          | 0/1752 [00:00<?, ?it/s]

In [18]:
df.sample(5)

,City,Lat,Lon,Zipcode
1546,Chicago,41.802314,-87.734268,60632
1396,Chicago,41.944535,-87.654764,60657
1054,Chicago,41.848470,-87.685109,60608
656,Chicago,41.701234,-87.649300,60643
1438,Chicago,41.651902,-87.539559,60633


In [21]:
df[df['City'] == 'Denver'].head()

,City,Lat,Lon,Zipcode
1632,Denver,39.755674,-104.990665,80205
1633,Denver,39.758272,-104.987299,80205
1634,Denver,39.760428,-104.983099,80205
1635,Denver,39.754546,-104.978054,80205
1636,Denver,39.746095,-104.979630,80203
